
# Making images of labeling patterns

The following notebook generates images of labeling patterns from MSI data acquired on a stable isotopically labeled sample. The input for this analysis is a folder of .imzML files (with associated .idb files) and a metaspace output file that denotes which metabolites to look for. The output of the notebook is natural abundance corrected isotopologue images saved in .png format in a folder named after each file. To run this notebook, enter the parameters in the cell below and upload files in the panel to the left. All cells should be run by clicking in the upper left corner of reach cell or pressing shift + enter. Cells should be run in order. After running the second cell, an example .imzml file from the SIMSIToolBox GitHub page (https://github.com/e-stan/imaging) will be downloaded. Please contact Ethan (gjpcolab@gmail.com) if you encounter any issues. 



In [1]:
imzMLDir = "X:/MSI_Shared_Data/example_data/labeled/" #path to folder that contains .imzML and .idb files
metaspaceFile = "X:/MSI_Shared_Data/example_data/metaspace_annotations.csv" #path to csv file that is the output from metaspace
correctNA = True #whether to perform natural abundance correction or not
segment = True #whether to segment and remove background
smooth = True #whether to smooth data

ppmThresh = 20 #m/z tolerance, all peaks within ppmThresh ppm will be summed into one feature
convSquare = 3 #size of filter (1=1x1,3=3x3,5=5x5)
colormap = "inferno" #coloring for images, see https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
num_cores = 20 #number of processor cores to use
dm_method = "PCA" #method for dimensionality reduction ("PCA" or "TSNE") PCA has worked better for me
seg_method = "TIC_auto" #thresholding method ("TIC_auto", "K_means", "TIC_manual")
num_components = 2 #number of compoents to use with PCA or TSNE
filt = "GB" #filtering method (GB = gaussian blur, MA = moving average)
intensityCutoff = 100 #intensity cutoff for considering peaks

In [2]:
#load libraries
import SIMSIToolBox
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.dpi'] = 100
import zipfile
import os
import pandas as pd
import numpy as np


In [3]:
#read metaspace file
metaspace = pd.read_csv(metaspaceFile,header=2) 
metaspace

,group,datasetName,datasetId,formula,adduct,chemMod,ion,mz,msm,fdr,...,moleculeNames,moleculeIds,minIntensity,maxIntensity,totalIntensity,isomers,isobars,offSample,rawOffSampleProb,isobarIons
0,NaN,40um-NEDC-maldi1,2022-09-23_14h13m33s,C4H6O4,M-H,NaN,C4H6O4-H-,117.019254,0.094677,0.20,...,"Erythrono-1,4-lactone, Methylmalonic acid, Suc...","HMDB0000349, HMDB0000202, HMDB0000254, HMDB000...",0,1222,1809174,7,1,False,0.037365,C5H8OS-H-
1,NaN,40um-NEDC-maldi1,2022-09-23_14h13m33s,C8H8O,M-H,NaN,C8H8O-H-,119.050160,0.070906,0.20,...,"4-Hydroxystyrene, Phenylacetaldehyde, 2,3-Dihy...","HMDB0004072, HMDB0006236, HMDB0013815, HMDB002...",0,510,409673,10,0,False,0.000045,NaN
2,NaN,40um-NEDC-maldi1,2022-09-23_14h13m33s,C2H7NO3S,M-H,NaN,C2H7NO3S-H-,124.007309,0.654689,0.05,...,Taurine,HMDB0000251,0,64368,1072943616,1,0,False,0.000191,NaN
3,NaN,40um-NEDC-maldi1,2022-09-23_14h13m33s,C5H6N2O2,M-H,NaN,C5H6N2O2-H-,125.035573,0.075157,0.20,...,"1H-Imidazole-1-acetic acid, Thymine, Imidazole...","HMDB0029736, HMDB0000262, HMDB0002024",0,1074,1970451,3,1,False,0.000011,C6H9NS-H-
4,NaN,40um-NEDC-maldi1,2022-09-23_14h13m33s,C6H9NO2,M-H,NaN,C6H9NO2-H-,126.055974,0.080558,0.20,...,"D-1-Piperideine-2-carboxylic acid, (S)-2,3,4,5...","HMDB0001084, HMDB0012130, HMDB0029434, HMDB005...",0,1006,5709366,4,1,False,0.000108,C5H7N3O-H-
5,NaN,40um-NEDC-maldi1,2022-09-23_14h13m33s,C5H8N2O2,M-H,NaN,C5H8N2O2-H-,127.051223,0.082143,0.20,...,"Dihydrothymine, Squamolone, L-Cyclo(alanylglyc...","HMDB0000079, HMDB0029874, HMDB0031547, HMDB006...",0,1774,6136436,4,1,False,0.000015,C9H7N-H-
6,NaN,40um-NEDC-maldi1,2022-09-23_14h13m33s,C5H7NO3,M-H,NaN,C5H7NO3-H-,128.035238,0.116014,0.20,...,"Pyroglutamic acid, Pyrrolidonecarboxylic acid,...","HMDB0000267, HMDB0000805, HMDB0001369, HMDB000...",0,2454,21926916,7,2,False,0.000084,"C6H10OS-H-, C6H9NS-H-"
7,NaN,40um-NEDC-maldi1,2022-09-23_14h13m33s,C6H14N2O,M-H,NaN,C6H14N2O-H-,129.103258,0.096482,0.20,...,N-Acetylputrescine,HMDB0002064,0,923,1327657,1,1,False,0.000009,C7H16O2-H-
8,NaN,40um-NEDC-maldi1,2022-09-23_14h13m33s,C5H9NO3,M-H,NaN,C5H9NO3-H-,130.050888,0.133482,0.20,...,"4-Hydroxy-L-proline, 4-Hydroxyproline, N-Acety...","HMDB0006055, HMDB0000725, HMDB0000766, HMDB000...",0,3100,22197690,13,1,False,0.000495,C7H7N3-H-
9,NaN,40um-NEDC-maldi1,2022-09-23_14h13m33s,C4H7NO4,M-H,NaN,C4H7NO4-H-,132.030153,0.087040,0.20,...,"L-Aspartic acid, Iminodiacetic acid, D-Asparti...","HMDB0000191, HMDB0011753, HMDB0006483, HMDB006...",0,3084,24953270,4,0,False,0.000665,NaN


In [4]:
#get imzMLs in folder
imzMLs = [x for x in os.listdir(imzMLDir) if ".imzML" in x] #get imzMLs
imzMLs

['4-1_10J-H_nedc_12x_50um_recal.imzML', '4-2_10F-I_nedc_50um_12x_recal.imzML']

In [5]:
#get m/zs of all ions
mzs = []
keys = []
for index,row in metaspace.iterrows():
    _,_,nC = SIMSIToolBox.getMzsOfIsotopologues(row["formula"],"C")
    for x in range(nC+1):
        mzs.append(row["mz"] + 1.00336 * x)
        keys.append((index,x))
print(len(mzs))

inds = []
for index,row in metaspace.iterrows():
    tmp = [x for x in range(len(keys)) if keys[x][0] == index]
    tmp.sort(key=lambda x: keys[x][1])
    inds.append(tmp)
metaspace["inds"] = inds

65


In [6]:
#load msi data
if __name__ == "__main__":
    msis = {}
    for file in imzMLs:
        msi = SIMSIToolBox.MSIData(mzs,ppm=ppmThresh,numCores = num_cores,intensityCutoff=intensityCutoff)
        msi.readimzML(imzMLDir + file)
        if segment:
            msi.segmentImage(method=seg_method, num_latent=num_components, dm_method=dm_method,fill_holes = True)
            plt.close()
        if smooth:
            msi.smoothData(filt,convSquare)
        if correctNA:
            msi.correctNaturalAbundance(metaspace["formula"].values,metaspace["inds"].values)
        msis[file] = msi



extracting intensities |██████████████████████████████████████████████████| 100.0% 
Smoothing data |██████████████████████████████████████████████████| 100.0% 
correcting natural abundance |██████████████████████████████████████████████████| 100.0% 
extracting intensities |██████████████████████████████████████████████████| 100.0% 
Smoothing data |██████████████████████████████████████████████████| 100.0% 
correcting natural abundance |██████████████████████████████████████████████████| 100.0% 


In [8]:
#make output folders
for fn in msis:
    try: os.mkdir(imzMLDir + fn.replace(".imzML",""))
    except: pass
colormapper = matplotlib.colormaps[colormap]
#iterate over each compound
for index,row in metaspace.iterrows():
    #get indices for compound
    inds = row["inds"]
    #iterate over files
    for fn in msis:
        #get signal
        tmp = msis[fn].data_tensor[inds]
        isoTensor = SIMSIToolBox.normalizeTensor(tmp)
        maxLim =np.max(isoTensor)
        counter = 0
        #plot images
        for img in isoTensor:
            norm = plt.Normalize(vmin=0, vmax=maxLim)
            norm = colormapper(norm(img))
            for r in range(norm.shape[0]):
                for c in range(norm.shape[1]):
                    if msis[fn].imageBoundary[r,c] < .5:
                        norm[r,c,3] = 0
            plt.imsave(imzMLDir + fn.replace(".imzML","/") + row["formula"] + "_M" + str(counter) +".png",norm)
            plt.close()
            plt.figure()
            SIMSIToolBox.showImage(img,cmap = colormapper)
            plt.clim(0,maxLim)
            plt.savefig(imzMLDir + fn.replace(".imzML","/") + row["formula"] + "_M" + str(counter) +"_withColorBar.png")
            plt.close()
            counter += 1
       
        #plot pool size images
        poolSize = np.sum(tmp,axis=0)
        meanInt = np.mean(msis[fn].tic_image[msis[fn].imageBoundary > 0.5])
        poolSize = poolSize / msis[fn].tic_image
        poolSize = poolSize * meanInt
        poolSize[msis[fn].imageBoundary < 0.5] = 0.0
        plt.figure()

        SIMSIToolBox.showImage(poolSize,cmap = colormapper)
        maxLim = np.max(poolSize)
        plt.clim(0,maxLim)
        norm = plt.Normalize(vmin=0, vmax=maxLim)
        norm = colormapper(norm(poolSize))
        for r in range(norm.shape[0]):
            for c in range(norm.shape[1]):
                if msis[fn].imageBoundary[r,c] < .5:
                    norm[r,c,3] = 0
        plt.imsave(imzMLDir + fn.replace(".imzML","/") + row["formula"] + "_pool.png",norm)
        plt.savefig(imzMLDir + fn.replace(".imzML","/") + row["formula"] + "_pool_with_colorbar.png")
        plt.close()



c:\users\ethan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


# Change log

03/19/22 notebook updated (ES)